# Weather Underground Web API


El siguiente Python Notebook muestra cómo obtener datos meteorológicos de cualquier aeropuerto del mundo, en formato horario.<br>
Estos datos históricos proceden de <b>Weather Underground</b> (https://www.wunderground.com/).<br><br>
Para configurar la búsqueda, es necesario especificar tres campos:<br>
<li>api-key: Se puede obtener en el siguiente [enlace](https://espanol.wunderground.com/weather/api).</li>
<li>query-date: Fecha de extracción deseada en formato *'YYYYMMDD'*.</li>
<li>IATA_code: Código del aeropuerto, disponible [aquí](https://www.world-airport-codes.com/).</li>




In [314]:
import pandas as pd
import requests
import json
from datetime import datetime as dt

In [315]:
#Wunderground API Key
api_key = '****************'

In [316]:
#Format YYYYMMDD
query_date = '20180203'

In [317]:
#Airport Code
IATA_code = 'LEBL'

In [318]:
#url para extraer datos históricos horarios en función de la fecha y del aeropuerto
url = 'http://api.wunderground.com/api/' + api_key + '/history_' + query_date + '/q/' + IATA_code + '.json'

In [319]:
data = requests.get(url)

In [320]:
data.content

b'\n{\n  "response": {\n  "version":"0.1",\n  "termsofService":"http://www.wunderground.com/weather/api/d/terms.html",\n  "features": {\n  "history": 1\n  }\n\t}\n\t\t,\n\t"history": {\n\t\t"date": {\n\t\t"pretty": "February 3, 2018",\n\t\t"year": "2018",\n\t\t"mon": "02",\n\t\t"mday": "03",\n\t\t"hour": "00",\n\t\t"min": "00",\n\t\t"tzname": "Europe/Madrid"\n\t\t},\n\t\t"utcdate": {\n\t\t"pretty": "February 2, 2018",\n\t\t"year": "2018",\n\t\t"mon": "02",\n\t\t"mday": "02",\n\t\t"hour": "23",\n\t\t"min": "00",\n\t\t"tzname": "UTC"\n\t\t},\n\t\t"observations": [\n\t\t{\n\t\t"date": {\n\t\t"pretty": "12:00 AM CET on February 03, 2018",\n\t\t"year": "2018",\n\t\t"mon": "02",\n\t\t"mday": "03",\n\t\t"hour": "00",\n\t\t"min": "00",\n\t\t"tzname": "Europe/Madrid"\n\t\t},\n\t\t"utcdate": {\n\t\t"pretty": "11:00 PM GMT on February 02, 2018",\n\t\t"year": "2018",\n\t\t"mon": "02",\n\t\t"mday": "02",\n\t\t"hour": "23",\n\t\t"min": "00",\n\t\t"tzname": "UTC"\n\t\t},\n\t\t"tempm":"5", "tempi":"41

In [321]:
contenido = json.loads(data.content)
print(contenido)

{'response': {'version': '0.1', 'termsofService': 'http://www.wunderground.com/weather/api/d/terms.html', 'features': {'history': 1}}, 'history': {'date': {'pretty': 'February 3, 2018', 'year': '2018', 'mon': '02', 'mday': '03', 'hour': '00', 'min': '00', 'tzname': 'Europe/Madrid'}, 'utcdate': {'pretty': 'February 2, 2018', 'year': '2018', 'mon': '02', 'mday': '02', 'hour': '23', 'min': '00', 'tzname': 'UTC'}, 'observations': [{'date': {'pretty': '12:00 AM CET on February 03, 2018', 'year': '2018', 'mon': '02', 'mday': '03', 'hour': '00', 'min': '00', 'tzname': 'Europe/Madrid'}, 'utcdate': {'pretty': '11:00 PM GMT on February 02, 2018', 'year': '2018', 'mon': '02', 'mday': '02', 'hour': '23', 'min': '00', 'tzname': 'UTC'}, 'tempm': '5', 'tempi': '41', 'dewptm': '-6', 'dewpti': '21', 'hum': '32', 'wspdm': '13.0', 'wspdi': '8.1', 'wgustm': '', 'wgusti': '', 'wdird': '0', 'wdire': 'North', 'vism': '30', 'visi': '19', 'pressurem': '1009', 'pressurei': '29.81', 'windchillm': '-999', 'windch

In [322]:
df_origin = pd.DataFrame(contenido)
df_origin

,history,response
dailysummary,[{'date': {'pretty': '12:00 AM CET on February...,NaN
date,"{'pretty': 'February 3, 2018', 'year': '2018',...",NaN
features,NaN,{'history': 1}
observations,[{'date': {'pretty': '12:00 AM CET on February...,NaN
termsofService,NaN,http://www.wunderground.com/weather/api/d/term...
utcdate,"{'pretty': 'February 2, 2018', 'year': '2018',...",NaN
version,NaN,0.1


Es necesario utilizar la función pandas.io.json.json_normalize para "normalizar" los datos json semiestructurados en una tabla plana. <br>
Puesto que los datos de interés se encuentran en la fila "observations", se aplica la función a dicha fila.

In [323]:
df_obs = pd.io.json.json_normalize(contenido['history']['observations'])
df_obs.head(5)

,conds,date.hour,date.mday,date.min,date.mon,date.pretty,date.tzname,date.year,dewpti,dewptm,...,visi,vism,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
0,,00,03,00,02,"12:00 AM CET on February 03, 2018",Europe/Madrid,2018,21,-6,...,19,30,0,North,,,-999,-999,8.1,13.0
1,Clear,00,03,00,02,"12:00 AM CET on February 03, 2018",Europe/Madrid,2018,21.2,-6.0,...,-9999.0,-9999.0,320,NW,-9999.0,-9999.0,34.3,1.3,11.5,18.5
2,Clear,00,03,30,02,"12:30 AM CET on February 03, 2018",Europe/Madrid,2018,21.2,-6.0,...,-9999.0,-9999.0,320,NW,-9999.0,-9999.0,32.0,0.0,11.5,18.5
3,Clear,01,03,00,02,"1:00 AM CET on February 03, 2018",Europe/Madrid,2018,21.2,-6.0,...,-9999.0,-9999.0,320,NW,-9999.0,-9999.0,31.6,-0.2,12.7,20.4
4,Clear,01,03,30,02,"1:30 AM CET on February 03, 2018",Europe/Madrid,2018,19.4,-7.0,...,-9999.0,-9999.0,330,NNW,-9999.0,-9999.0,36.4,2.5,6.9,11.1


Se estudian las series de interés en el DataFrame. Para ello, primeramente hay que saber qué series contiene el DataFrame

In [324]:
df_obs.columns

Index(['conds', 'date.hour', 'date.mday', 'date.min', 'date.mon',
       'date.pretty', 'date.tzname', 'date.year', 'dewpti', 'dewptm', 'fog',
       'hail', 'heatindexi', 'heatindexm', 'hum', 'icon', 'metar', 'precipi',
       'precipm', 'pressurei', 'pressurem', 'rain', 'snow', 'tempi', 'tempm',
       'thunder', 'tornado', 'utcdate.hour', 'utcdate.mday', 'utcdate.min',
       'utcdate.mon', 'utcdate.pretty', 'utcdate.tzname', 'utcdate.year',
       'visi', 'vism', 'wdird', 'wdire', 'wgusti', 'wgustm', 'windchilli',
       'windchillm', 'wspdi', 'wspdm'],
      dtype='object')

Se crea un nuevo DataFrame llamado "df_filter" formado por las series de "df_obs" que son de interés. En este caso se escogen "date.mday", "date.mon", "date.year", "date.hour", "date.min", "tempm", "pressure" y "hum".

In [325]:
df_filter = df[['date.mday', 'date.mon', 'date.year', 'date.hour', 'date.min', 'tempm', 'pressurem', 'hum']]
df_filter.head(5)

,date.mday,date.mon,date.year,date.hour,date.min,tempm,pressurem,hum
0,03,02,2018,00,00,5,1009,32
1,03,02,2018,00,00,5.0,1009,45
2,03,02,2018,00,30,4.0,1009,49
3,03,02,2018,01,00,4.0,1009,49
4,03,02,2018,01,30,5.0,1009,42


Se crea la serie 'date' y se añade al DataFrame "df_filter". Esta nueva serie se crea a partir de la concatenación de las series relativas a la fecha y a la hora, convirtiéndolas a formato datetime.

In [326]:
df_filter['date'] = pd.to_datetime(df['date.mday'] + '/' + df['date.mon'] + '/' + df['date.year'] + ' ' + df['date.hour'] + ':' + df['date.min'], format = '%d/%m/%Y %H:%M')

df_filter.head(5)

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,date.mday,date.mon,date.year,date.hour,date.min,tempm,pressurem,hum,date
0,03,02,2018,00,00,5,1009,32,2018-02-03 00:00:00
1,03,02,2018,00,00,5.0,1009,45,2018-02-03 00:00:00
2,03,02,2018,00,30,4.0,1009,49,2018-02-03 00:30:00
3,03,02,2018,01,00,4.0,1009,49,2018-02-03 01:00:00
4,03,02,2018,01,30,5.0,1009,42,2018-02-03 01:30:00


Se crea el nuevo DataFrame "df_dt" en el que se han eliminado las columnas relativas a la fecha y a la hora que no estaban en formato datetime.

In [327]:
df_dt = df_filter[['date', 'tempm', 'pressurem', 'hum']]
df_dt.head(5)

,date,tempm,pressurem,hum
0,2018-02-03 00:00:00,5,1009,32
1,2018-02-03 00:00:00,5.0,1009,45
2,2018-02-03 00:30:00,4.0,1009,49
3,2018-02-03 01:00:00,4.0,1009,49
4,2018-02-03 01:30:00,5.0,1009,42


Se renombran las columnas del dataframe "df_dt".

In [328]:
df_dt.columns = ['Fecha', 'Temperatura', 'Presion', 'Humedad']

Se forma un nuevo DataFrame "df1" formado por los datos existentes del DataFrame "df_dt". <br>
Para ello, se crean dos máscaras booleanas. La primera de ellas servirá para filtrar aquellos datos que tengan una hora exacta, mientras que la segunda máscará tendrá como finalidad quedarse con aquellos datos que tengan mayor exactitud (los que contienen ".").

In [329]:
mascara_hora_exacta = df_dt['Fecha'].dt.minute == 0
mascara_contiene_punto = df_dt['Temperatura'].str.contains('.0')

df1 = df_dt[mascara_hora_exacta & mascara_contiene_punto]
df1.head(5)

,Fecha,Temperatura,Presion,Humedad
1,2018-02-03 00:00:00,5.0,1009,45
3,2018-02-03 01:00:00,4.0,1009,49
6,2018-02-03 02:00:00,5.0,1009,45
9,2018-02-03 03:00:00,5.0,1009,42
12,2018-02-03 04:00:00,5.0,1009,45


Se crea una nueva serie formada por la longitud de la cadena existente en la serie "Temperatura".

In [330]:
df1['Longitud'] = df1['Temperatura'].str.len()
df1.sample(5)

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Fecha,Temperatura,Presion,Humedad,Longitud
6,2018-02-03 02:00:00,5.0,1009,45,3
63,2018-02-03 21:00:00,10.0,1010,54,4
30,2018-02-03 10:00:00,6.0,1013,53,3
59,2018-02-03 20:00:00,10,1011,45,2
3,2018-02-03 01:00:00,4.0,1009,49,3


Se ordenan las series por fecha y longitud, de manera descendente, para posteriormente eliminar las fechas duplicadas.

In [331]:
df2 = df1.sort_values(by = ['Fecha', 'Longitud'], ascending= False)
df2 = df2.drop_duplicates(subset = 'Fecha')
df2 = df2.sort_values(by = 'Fecha')
df2 = df2.reset_index(drop = True)

Se crea el DataFrame "df_final" en el que se encuentran los resultados filtrados y ordenados, y se ha eliminado la serie "Longitud".

In [332]:
df_final = df2.drop('Longitud', axis = 1)

Finalmente, los resultados extraídos mediante la consulta son los siguientes:

In [333]:
df_final

,Fecha,Temperatura,Presion,Humedad
0,2018-02-03 00:00:00,5.0,1009,45
1,2018-02-03 01:00:00,4.0,1009,49
2,2018-02-03 02:00:00,5.0,1009,45
3,2018-02-03 03:00:00,5.0,1009,42
4,2018-02-03 04:00:00,5.0,1009,45
5,2018-02-03 05:00:00,5.0,1010,45
6,2018-02-03 06:00:00,5.0,1010,45
7,2018-02-03 07:00:00,4.0,1010,52
8,2018-02-03 08:00:00,4.0,1011,52
9,2018-02-03 09:00:00,4.0,1012,56
